## Using Google's Agent Development Kit (ADK) with Neo4j MCP

This notebook demonstrates how to use Google's Agent Development Kit (ADK) to create a conversational agent that can query a Neo4j database. We will configure an LLM-based agent, connect it to a Neo4j database using the Model Context Protocol (MCP), and then ask it a question about the data.

### Installation

This cell installs the necessary Python libraries:
- `google-adk`: The Google Agent Development Kit.

The `-q` flag ensures a quiet installation, minimizing the output.

In [ ]:
%pip install -q google-adk

### Imports

This cell imports all the required modules for the notebook. Grouping imports at the beginning of a script or notebook is a standard practice that improves code readability and helps manage dependencies.

In [2]:
import os
import json
from getpass import getpass
from neo4j import GraphDatabase
from google.genai import types
from google.adk.agents.llm_agent import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService
from google.adk.tools.mcp_tool import McpToolset
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

We will use gemini flash latest for this example, but you can choose any Gemini model that supports tool use. Check the ADK documentation for the latest supported models and their capabilities.

In [3]:
os.environ["GOOGLE_API_KEY"] = getpass("Enter Google API Key: ")
os.environ["GEMINI_MODEL"] = "gemini-flash-latest"

# Neo4j MCP Server
We'll start by using the official [Neo4j MCP Server](https://github.com/neo4j/mcp) to extend the agent with Neo4j tools. This MCP server provides the agent with capabilities to read the graph schema and execute Cypher queries, enabling it to fetch and analyze data directly from the database.

The following code installs the latest version on Google Colab and similar Linux-based systems. For other operating systems, please consult the [official installation documentation](https://neo4j.com/docs/mcp/current/installation/).

In [4]:
import requests

# Get latest release info from GitHub API
release = requests.get("https://api.github.com/repos/neo4j/mcp/releases/latest").json()
version = release["tag_name"]
print(f"Latest version: {version}")

# Download the latest Linux binary
!wget -q https://github.com/neo4j/mcp/releases/download/{version}/neo4j-mcp_Linux_x86_64.tar.gz

# Extract
!tar -xzf neo4j-mcp_Linux_x86_64.tar.gz

# Make executable
!sudo chmod +x neo4j-mcp

# Cleanup
!rm neo4j-mcp_Linux_x86_64.tar.gz

# Move
!sudo mv neo4j-mcp /usr/local/bin/

# Verify installation
!neo4j-mcp -v

Latest version: v1.3.0
neo4j-mcp version: v1.3.0


For this example, we'll use the companies database from the Neo4j demo server, which contains organizations, people, investors, and news articles.m

In [5]:
os.environ["NEO4J_URI"] = "neo4j+s://demo.neo4jlabs.com"
os.environ["NEO4J_USERNAME"] = "companies"
os.environ["NEO4J_PASSWORD"] = "companies"
os.environ["NEO4J_DATABASE"] = "companies"

### Initialize MCP Toolset
We will create the MCP toolset, which allows the agent to interact with the Neo4j database via the MCP server. The toolset is configured with connection parameters and environment variables for secure access.

In [6]:
mcp_tools  = McpToolset(
        connection_params=StdioConnectionParams(
            server_params=StdioServerParameters(
                command='neo4j-mcp', 
                args=[], 
                env={
                    "NEO4J_URI": os.environ["NEO4J_URI"],
                    "NEO4J_USERNAME": os.environ["NEO4J_USERNAME"],
                    "NEO4J_PASSWORD": os.environ["NEO4J_PASSWORD"],
                    "NEO4J_DATABASE": os.environ["NEO4J_DATABASE"],
                    "NEO4J_READ_ONLY": "true" # Optional safety flag
                }
            )
        )
    )

### Agent definition

we will define an asynchronous function to create an ADK agent. The function takes model, name, prompt, and tools as parameters, returning a configured LLM agent instance. This modular approach supports flexible agent creation.

In [7]:
async def create_agent(model, name, prompt, toolsList):
    """Creates an ADK Agent """
    
    root_agent = LlmAgent(
        model=model,
        name=name,
        instruction=prompt,
        tools=toolsList
    )
    return root_agent

### Create Neo4j Explorer Agent
we will create an agent which will use mcp tools and query database.

In [8]:
prompt = """You are a graph database assistant. Use the available tools to query the Neo4j database. 
        If you are unsure of the schema, run 'get-schema' first.
        """
model = os.environ["GEMINI_MODEL"]
name = "neo4j_explorer"
toolsList = [mcp_tools] 
mcp_agent = await create_agent(model, name, prompt, toolsList)  

### Run the agent

In [9]:
async def run_agent(agent, app_name, user_id, query):
    session_service = InMemorySessionService()
    artifacts_service = InMemoryArtifactService()

    session = await session_service.create_session(
        state={}, app_name=app_name, user_id=user_id
    )


    print(f"User Query: '{query}'")

    content = types.Content(role='user', parts=[types.Part(text=query)])

    runner = Runner(
        app_name=app_name,
        agent=agent,
        artifact_service=artifacts_service,
        session_service=session_service,
    )

    try:
        events_async = runner.run_async(
            session_id=session.id, 
            user_id=session.user_id, 
            new_message=content
        )

        async for event in events_async:
            if hasattr(event, 'content') and event.content:
                for part in event.content.parts:
                    if hasattr(part, 'text') and part.text:
                        print(f"\nFinal Answer: {part.text}\n")
            # for debugging or insight into the agent's thought process, you can print all events:
            # else:
            #     print(f"Progress: {event}")
    except Exception as e:
        print(f"Error during agent execution: {e}")


In [10]:
query = "How many people are in the database?"
app_name = "neo4j_explorer_app"
user_id = "user_123"
await run_agent(mcp_agent, app_name, user_id, query)

User Query: 'How many people are in the database?'



Final Answer: There are 8,064 people in the database.



# Custom tools

Beyond using existing MCP servers, you can also implement your own custom tools and add them directly to the agent. This allows you to create specialized functionality tailored to your specific use case. Custom tools can be implemented using the @tool decorator, which turns any function into a tool the agent can invoke.

Here, we use GraphDatabase from the neo4j package to establish a connection to our database and build a tool that queries investment relationships, giving you more control over the query logic.

In [11]:
async def get_investments(company: str) -> str:
    """
    Returns the investments by a company by name. 
    Returns list of investment ids, names and types.
    """
    query = """
    MATCH (o:Organization)-[:HAS_INVESTOR]->(i)
    WHERE o.name = $company
    RETURN i.id as id, i.name as name, head(labels(i)) as type
    """
    try:
        driver = GraphDatabase.driver(os.environ["NEO4J_URI"], auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]), database=os.environ["NEO4J_DATABASE"])
        records, _, _ = driver.execute_query(
            query, 
            company=company
        )
        results = [record.data() for record in records]
        return json.dumps(results, indent=2)
    except Exception as e:
        return f"Error fetching investments: {str(e)}"


Transforming a Python function into a tool is a straightforward way to integrate custom logic into your agents. When you assign a function to an agent’s tools list, the framework automatically wraps it as a FunctionTool.

In [12]:
custom_investment_tool = FunctionTool(get_investments)

We will combine the mcp tools and custom tool for our custom agent

In [13]:
custom_tools = [mcp_tools, custom_investment_tool]
custom_agent_prompt = "You are a helpful assistant with access to a Neo4j graph database containing company data. Use the available tools to query the database and answer questions."
custom_agent_name = "Custom_neo4j_agent"
custom_agent = await create_agent(model, custom_agent_name, custom_agent_prompt, custom_tools)

In [14]:
query = "Which companies did Google invest in?"
user_id = "user_123"
await run_agent(custom_agent, custom_agent_name, user_id, query)

User Query: 'Which companies did Google invest in?'

Final Answer: Google invested in the following companies:

*   Ionic Security
*   Avere Systems
*   FlexiDAO
*   Cloudflare
*   Trifacta



# Summary

This notebook demonstrates how to use Google's Agent Development Kit (ADK) to build conversational agents that interact with a Neo4j graph database. The workflow includes:

- Connecting to the Neo4j MCP server for schema and Cypher query access.
- Defining and running an LLM agent with MCP tools.
- Implementing and integrating custom tools for specialized queries.
- Running agents to answer questions using both MCP and custom tools.
